# Gaining insights from Unstructured Data with Snowflake Cortex AI

Tasty Bytes is a global food truck network operating in 15 countries with fleet of 450 trucks. They collect customer reviews to get customer feedback on their food-trucks which come in from multiple sources and span multiple languages. 
This enables them to better understand the areas which require improvement and drive up customer loyalty along with satisfaction. 

In this notebook, we will look at how we analyze these collated customer reviews using Snowflake Cortex to understand :
  * What our international customers are saying with Cortex **Translate**
  * Get a summary of what customers are saying with Cortex **Summary**
  * Classify reviews to determine if they would recommend a food truck with Cortex **ClassifyText**
  * Gain specific insights with Cortex **Complete**
  * Understand how customers are feeling with Cortex **Sentiment**

Lets see how many reviews we have.

In [ ]:
SELECT COUNT(*) FROM TRUCK_REVIEWS_V;

**Import python packages**

Snowflake Notebooks include Streamlit and the third-party packages listed in the Snowflake Anaconda channel.  
Installing a package is made easy by enabling user to select required pacakges from a list of available pacakges under Packages on the top right corner.  
Once installed, we can import installed packages as we would in any other notebook.

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# Snowpark
from snowflake.snowpark.context import get_active_session
import snowflake.snowpark.functions as F
from snowflake.snowpark.functions import when, date_part
from snowflake.snowpark.window import Window

# Cortex Functions
import snowflake.cortex  as cortex

session = get_active_session()
# Add a query tag to the session.
session.query_tag = {"origin":"sf_sit-is", 
                     "name":"voc", 
                     "version":{"major":1, "minor":0},
                     "attributes":{"is_quickstart":1, "source":"notebook", "vignette":"customer_reviews"}}


Lets preview the reviews

In [ ]:
reviews_df = session.table('TRUCK_REVIEWS_V') \
             .filter(date_part("year", F.col('DATE')) == 2024)
reviews_df.show()

In the next cell, we leverage **Translate** - one of the **Snowflake Cortex specialised LLM functions** are available in Snowpark ML, to translate the multilingual reviews to english to enable easier analysis for folks who don't speak the language the original review is in.

In [ ]:
# Conditionally translate reviews that are not english using Cortex Translate
reviews_df = reviews_df.withColumn('TRANSLATED_REVIEW',when(F.col('LANGUAGE') != F.lit("en"), \
                                                            cortex.Translate(F.col('REVIEW'), \
                                                                             F.col('LANGUAGE'), \
                                                                             "en")) \
                                   .otherwise(F.col('REVIEW')))

reviews_df.filter(F.col('LANGUAGE') != F.lit("en")) \
.select(["REVIEW","LANGUAGE","TRANSLATED_REVIEW"]).show(3)

We can quickly learn what our customers are saying with Cortex **Summarize**

In [ ]:
# Step 1: Add a row number for each review within each TRUCK_BRAND_NAME
window_spec = Window.partition_by("TRUCK_BRAND_NAME").order_by("REVIEW")
ranked_reviews_df = reviews_df.with_column(
    "ROW_NUM", F.row_number().over(window_spec)
)

# Step 2: Filter to include only the first 20 rows per TRUCK_BRAND_NAME to get a general idea
filtered_reviews_df = ranked_reviews_df.filter(F.col("ROW_NUM") <= 20)

# Step 3: Aggregate reviews by TRUCK_BRAND_NAME
aggregated_reviews_df = filtered_reviews_df.group_by("TRUCK_BRAND_NAME").agg(
    F.array_agg(F.col("REVIEW")).alias("ALL_REVIEWS")
)

# Step 4: Convert the array of reviews to a single string
concatenated_reviews_df = aggregated_reviews_df.with_column(
    "ALL_REVIEWS_TEXT", F.call_function("array_to_string", F.col("ALL_REVIEWS"), F.lit(' '))
)

# Step 5: Generate summaries for each truck brand
summarized_reviews_df = concatenated_reviews_df.with_column(
    "SUMMARY", cortex.Summarize(F.col("ALL_REVIEWS_TEXT"))
)

# Step 6: Display the results
summarized_reviews_df.select(["TRUCK_BRAND_NAME", "SUMMARY"]).show(3)

one_summary_row = summarized_reviews_df.limit(1).collect()
if one_summary_row:
    brand = one_summary_row[0]['TRUCK_BRAND_NAME']
    summary = one_summary_row[0]['SUMMARY']

    # Split the summary roughly in half
    half = len(summary) // 2
    split_index = summary[:half].rfind(' ')  # Find the last space before the halfway point
    if split_index == -1:
        split_index = half  # If no space found, split at halfway

    summary_part1 = summary[:split_index].strip()
    summary_part2 = summary[split_index:].strip()

    print(f"Truck Brand: {brand}")
    print(f"Summary (part 1): {summary_part1}")
    print(f"Summary (part 2): {summary_part2}")

We can similarly understand if a customer would recommend the food truck based on their review using Snowflake Cortex **ClassifyText**

In [ ]:
# Text description to understand whether a customer would recommend food truck based on their review 
text_description = """
Tell me based on the following food truck customer review, will they recommend the food truck to \
their friends and family?
"""

reviews_df = reviews_df.withColumn('RECOMMEND', cortex.ClassifyText(F.col('REVIEW'),["Likely","Unlikely","Unsure"], text_description)['label'])

reviews_df.select(["REVIEW","RECOMMEND"]).show(3)

Gaining specific insights through Cortex Complete

In [ ]:
# Step 1: Add a row number for each review within each TRUCK_BRAND_NAME
window_spec = Window.partition_by("TRUCK_BRAND_NAME").order_by("REVIEW")
ranked_reviews_df = reviews_df.with_column(
    "ROW_NUM", F.row_number().over(window_spec)
)

# Step 2: Filter to include only the first 20 rows per TRUCK_BRAND_NAME to get a general idea
filtered_reviews_df = ranked_reviews_df.filter(F.col("ROW_NUM") <= 20)

# Step 3: Aggregate reviews by TRUCK_BRAND_NAME
aggregated_reviews_df = filtered_reviews_df.group_by("TRUCK_BRAND_NAME").agg(
    F.array_agg(F.col("REVIEW")).alias("ALL_REVIEWS")
)

# Step 4: Convert the array of reviews to a single string
concatenated_reviews_df = aggregated_reviews_df.with_column(
    "ALL_REVIEWS_TEXT", F.call_function("array_to_string", F.col("ALL_REVIEWS"), F.lit(' '))
)

question = "What is the number one dish positively mentioned in the feedback?"

# Step 5: Answer our question
summarized_reviews_df = concatenated_reviews_df.with_column(
    "NUMBER_ONE_DISH", cortex.Complete("mistral-large2", 
                                       F.concat(F.lit(f'Context:'), 
                                                F.col("ALL_REVIEWS_TEXT"), 
                                                F.lit(f' Question:{question} Answer briefly and concisely and only name the dish:'))))

summarized_reviews_df.select("TRUCK_BRAND_NAME", "NUMBER_ONE_DISH").show(3)

Next we will look at another **task specific LLM function in Cortex - Sentiment**. We utilise sentiment function to understand customer's tone based on the review they provided. Sentiment return value between -1 and 1 such that -1 is the most negative while 1 is the most positive.  

In [ ]:
# Understand the sentiment of customer review using Cortex Sentiment
reviews_df = reviews_df.withColumn('SENTIMENT', cortex.Sentiment(F.col('REVIEW')))

reviews_df.select(["REVIEW","SENTIMENT"]).show(3)

These robust Cortex functions seamlessly integrate with SQL, enabling powerful data processing capabilities.

In [ ]:
-- Add the TRANSLATED_REVIEW column with conditional translation
WITH TRANSLATED_REVIEWS AS (
    SELECT 
        REVIEW,
        LANGUAGE,
        CASE 
            WHEN LANGUAGE != 'en' THEN SNOWFLAKE.CORTEX.TRANSLATE(REVIEW, LANGUAGE, 'en') 
            ELSE REVIEW
        END AS TRANSLATED_REVIEW
    FROM TRUCK_REVIEWS_V
)

-- Filter rows where the LANGUAGE is not English and select the desired columns
SELECT 
    REVIEW, 
    LANGUAGE, 
    TRANSLATED_REVIEW
FROM TRANSLATED_REVIEWS
WHERE LANGUAGE != 'en'
LIMIT 3;

In [ ]:
-- Add a row number for each review within each TRUCK_BRAND_NAME
WITH RANKED_REVIEWS AS (
    SELECT 
        *,
        ROW_NUMBER() OVER (PARTITION BY TRUCK_BRAND_NAME ORDER BY REVIEW) AS ROW_NUM
    FROM TRUCK_REVIEWS_V
),

-- Filter to include only the first 20 rows per TRUCK_BRAND_NAME
FILTERED_REVIEWS AS (
    SELECT *
    FROM RANKED_REVIEWS
    WHERE ROW_NUM <= 20
),

-- Aggregate reviews by TRUCK_BRAND_NAME
AGGREGATED_REVIEWS AS (
    SELECT 
        TRUCK_BRAND_NAME,
        ARRAY_AGG(REVIEW) AS ALL_REVIEWS
    FROM FILTERED_REVIEWS
    GROUP BY TRUCK_BRAND_NAME
),

-- Convert the array of reviews into a single string
CONCATENATED_REVIEWS AS (
    SELECT 
        TRUCK_BRAND_NAME,
        ARRAY_TO_STRING(ALL_REVIEWS, ' ') AS ALL_REVIEWS_TEXT
    FROM AGGREGATED_REVIEWS
),

-- Generate summaries for each truck brand
SUMMARIZED_REVIEWS AS (
    SELECT 
        TRUCK_BRAND_NAME,
        SNOWFLAKE.CORTEX.SUMMARIZE(ALL_REVIEWS_TEXT) AS SUMMARY
    FROM CONCATENATED_REVIEWS
)

-- Display the summaries and optionally split them into two parts
SELECT 
    TRUCK_BRAND_NAME,
    SUMMARY,
    LEFT(SUMMARY, FLOOR(LENGTH(SUMMARY) / 2)) AS SUMMARY_PART1,
    RIGHT(SUMMARY, LENGTH(SUMMARY) - FLOOR(LENGTH(SUMMARY) / 2)) AS SUMMARY_PART2
FROM SUMMARIZED_REVIEWS
LIMIT 3;


In [ ]:
WITH CLASSIFIED_REVIEWS AS (
    SELECT 
        REVIEW,
        PARSE_JSON(SNOWFLAKE.CORTEX.CLASSIFY_TEXT(
            REVIEW, 
            ['Likely', 'Unlikely', 'Unsure'], 
            OBJECT_CONSTRUCT('task_description', 
                'Tell me based on the following food truck customer review, will they recommend the food truck to their friends and family?'
            )
        )):label::TEXT AS RECOMMEND
    FROM TRUCK_REVIEWS_V
)

SELECT * From CLASSIFIED_REVIEWS limit 3;


In [ ]:
WITH RANKED_REVIEWS AS (
        SELECT 
            TRUCK_BRAND_NAME,
            REVIEW,
            ROW_NUMBER() OVER (PARTITION BY TRUCK_BRAND_NAME ORDER BY REVIEW) AS ROW_NUM
        FROM TRUCK_REVIEWS_V
    ),
    FILTERED_REVIEWS AS (
        SELECT *
        FROM RANKED_REVIEWS
        WHERE ROW_NUM <= 20
    ),
    AGGREGATED_REVIEWS AS (
        SELECT 
            TRUCK_BRAND_NAME,
            ARRAY_AGG(REVIEW) AS ALL_REVIEWS
        FROM FILTERED_REVIEWS
        GROUP BY TRUCK_BRAND_NAME
    ),
    CONCATENATED_REVIEWS AS (
        SELECT 
            TRUCK_BRAND_NAME,
            ARRAY_TO_STRING(ALL_REVIEWS, ' ') AS ALL_REVIEWS_TEXT
        FROM AGGREGATED_REVIEWS
    ),
    SUMMARIZED_REVIEWS AS (
        SELECT 
            TRUCK_BRAND_NAME,
            SNOWFLAKE.CORTEX.COMPLETE(
               'mistral-large2', 
               'Context:' || ALL_REVIEWS_TEXT || ' Question: What is the number one dish positively mentioned in the feedback? Answer briefly and concisely and only name the dish:'
           ) AS NUMBER_ONE_DISH
        FROM CONCATENATED_REVIEWS
    )
    SELECT TRUCK_BRAND_NAME, NUMBER_ONE_DISH FROM SUMMARIZED_REVIEWS LIMIT 3;

In [ ]:
SELECT 
    REVIEW, 
    SNOWFLAKE.CORTEX.SENTIMENT(REVIEW) AS SENTIMENT
FROM TRUCK_REVIEWS_V
LIMIT 3;
